# Outlier detection based on persistent homology

by Matthew J. Graham (California Institute of Technology/National Optical Astronomy Observatory)

(c) 2017

<b><i>Version: 0.1</i></b>

<i>An outlier is an observation that differs so much from other observations as to arouse suspicion that it was generated by a different mechanism</i> (Hawkins 1980).


## Introduction

Characterized distributions of features inform about population behaviors. The global <i>shape</i> of the data in the $n$-dimensional parameter space can provide information about the phenomena they represent. Planar projections fail to represent features of point data clouds where the space is too high dimensional or too twisted. 

<i>Persistent homology</i> is a technique from topological data analysis (TDA) which identifies which topological features (components, holes, graph structure) persist over a wide range of length scales. These are more likely to represent true features of the underlying space rather than artifacts of sampling, noise, or parameter choice. They will also be invariant to data perturbations.

From the perspective of TDA, outliers can be regarded as having minimal connectivity within a data space over a substantial range of scales. There are spatial constructs which can be used as proxies for more complex topological analysis techniques (persistent homology). The goal of this hack is to explore how well these work with astronomical feature spaces and to see what types of outliers they identify relative to more "traditional" outlier detection techniques, e.g., tails of distributions.

## Methodology

The <i>minimal spanning tree (MST)</i> for a data set is a unique construct which encodes all the relevant information about connected components at all resolutions. It is invariant under rotations, scalings, and translations. <i>Pruning</i> is equivalent to compact partitioning and produces clusters that are precisely persistent components. Algebraic topological measures can be derived from the MST, e.g., Betti numbers, and the asymptotic behaviors of the MST are well understood (mean-field limit approach). 

The MST will not just identify low density regions but also areas of low connectivity. In addition to looking for outliers (marginally connected points based on the node degree), <i>hub</i> nodes may also be of interest since these bridge clusters. This approach is non-parametric.

The number of components, $\beta_0(\alpha)$, is just one more than the number of MST edges that are longer than $2 \alpha$. Robins et al. (2000) show that the MST edge lengths have a close connectedness with $\epsilon-$connectedness and the $\epsilon$-components.

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Load the data

We're using test data from the <a href="https://github.com/fedhere/DtUhackOutliers.git">DtUhackOutliers</a> Git repo. 


In [3]:
#data = np.load('../DtUhackOutliers/data/KeplerSampleWErrSparse.npy')
data = np.genfromtxt("data.csv", delimiter = ",", skip_header = 1)

### Constructing the MST

We'll initially try using the MST algorithm in scipy (this will be an approximate MST since nearest neighbours are used) with a user-specified metric:

In [ ]:
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics import pairwise_distances

metric = "euclidean" # see the sklearn.DistanceMetric class for options
metric_params = None
n_neighbors = 100

# Calculate the k-nearest neighbors graph
#G = kneighbors_graph(data, n_neighbors = n_neighbors, mode = 'distance', 
#                     metric = metric, metric_params = metric_params)

# Full distance matrix
kwds = metric_params or {}
G = pairwise_distances(data, metric = metric, **kwds)

# Calculate the MST of this graph
MST = minimum_spanning_tree(G, overwrite = True)
# Might need to ensure that MST is symmetrical

### Identifying persistence

The Robins et al. (2000) work recasts the MST as a binary tree for efficiency. At this stage, though, we want to identify the persistence of each node rather than the overall number of $\epsilon-$components or their diameters. There is a minimum length scale, $l_{min}$, below which all nodes are separate and a maximum length scale, $l_{max}$, above which the MST is completely connected. We'll define the persistence of a node to be the fraction of $l_{max} - l_{min}$ for which the node is isolated, e.g., unconnected from any other node. We could also modify this to define persistence below a paricular degree level as well.

In [23]:
lmin = MST[MST > 0].min() # May need to check that this is a true value and not rounding error
lmax = MST[MST > 0].max()
persist = np.zeros(len(data))
degree = MST.getnnz(1)
for i in range(len(data)):
    # Approximate MST means that some nodes are not connected to MST
    if degree[i] > 0: persist[i] = (MST[i][MST[i] > 0.].min() - lmin) / (lmax - lmin)
    
idx = np.argsort(persist)

In [24]:
persist[idx]

array([ 0.        ,  0.        ,  0.        , ...,  0.00684453,
        0.99865848,  1.        ])

In [25]:
len(persist[persist == 0.])

17233